In [1]:
from datasets import load_dataset 

In [2]:
ds = load_dataset("eliplutchok/fineweb-small-sample", split='train')
ds = ds.select(range(min(300_000, len(ds))))

In [3]:
ds

Dataset({
    features: ['text', 'id', 'dump', 'url', 'file_path', 'language', 'language_score', 'token_count', 'score', 'int_score'],
    num_rows: 300000
})

In [4]:
from utils import tokenize 
import tiktoken 

tokenizer = tiktoken.get_encoding("gpt2")
tokens = tokenize(tokenizer,ds)

Tokenizing:   0%|          | 0/300000 [00:00<?, ?it/s]

In [ ]:
from sklearn.model_selection import train_test_split

# Split into train and test sets (70% train, 30% test)
train_tokens, test_tokens = train_test_split(tokens, test_size=0.3, random_state=42)


In [ ]:
import numpy as np
import torch
from tqdm import tqdm
from utils import tokenize 
import tiktoken 


def preprocess_and_save(token_ids, max_length, stride, save_path):
    inputs = []
    targets = []
    
    # Calculate the number of steps for tqdm
    num_steps = (len(token_ids) - max_length) // stride + 1
    
    # Preprocess token_ids with tqdm to show progress
    for i in tqdm(range(num_steps), desc=f"Processing {save_path}"):
        start_idx = i * stride
        input_chunk = token_ids[start_idx:start_idx + max_length]
        target_chunk = token_ids[start_idx + 1:start_idx + max_length + 1]
        inputs.append(input_chunk)
        targets.append(target_chunk)
    
    np.savez(save_path, inputs=np.array(inputs), targets=np.array(targets))


if __name__ == "__main__":
    tokenizer = tiktoken.get_encoding("gpt2")
    args = parse_args()

    split_and_save(tokens)


In [ ]:
import torch
import numpy as np

# Convert tokens to a PyTorch tensor or NumPy array for easier slicing
tokens_tensor = torch.tensor(tokens)
num_samples = len(tokens_tensor)

# Shuffle the data (if needed)
indices = torch.randperm(num_samples)

# Calculate split indices
split_idx = int(num_samples * 0.7)

# Split the dataset
train_tokens = tokens_tensor[indices[:split_idx]]
test_tokens = tokens_tensor[indices[split_idx:]]
